<a href="https://colab.research.google.com/github/Ramzy70/FreeCodeCamp-MachineLearning-Course/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the data
train_data = pd.read_csv("train-data.tsv", sep='\t', names=['label', 'message'])
test_data = pd.read_csv("valid-data.tsv", sep='\t', names=['label', 'message'])

In [ ]:
print(train_data.tail())

In [ ]:
# Preprocess the data
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train_data['message'])
X_test = vectorizer.transform(test_data['message'])

y_train = (train_data['label'] == 'spam').astype(int)
y_test = (test_data['label'] == 'spam').astype(int)

from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess the data using TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['message'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['message'])

# Split the data into training and validation sets
X_train_tfidf, X_val_tfidf, y_train_tfidf, y_val_tfidf = train_test_split(X_train_tfidf, y_train, test_size=0.1, random_state=42)


In [ ]:
# Build and train the model
model_tfidf = keras.Sequential([
    keras.layers.Dense(1024, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model_tfidf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history_tfidf = model_tfidf.fit(X_train_tfidf.toarray(), y_train_tfidf, epochs=10, batch_size=128, validation_data=(X_val_tfidf.toarray(), y_val_tfidf))

In [ ]:
# Test the predictions
test_predictions_tfidf()

In [ ]:
# Evaluate model on test data
test_loss, test_accuracy = model_tfidf.evaluate(X_test.toarray(), y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
# Define the function to predict messages based on the model
def predict_message(pred_text):
    pred_text_transformed = vectorizer.transform([pred_text])
    prediction = model_tfidf.predict(pred_text_transformed.toarray())[0][0]
    label = "spam" if prediction >= 0.5 else "ham"
    return [prediction, label]

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(msg , prediction , prediction[1] == ans)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
